# 기본 설정

구글 드라이브 마운트

패키지 설치

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
!pip install cinemagoer
!pip install transformers
!pip install datasets
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 301 kB 8.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 7.3 MB/s 
     |████████████████████████████████| 86 kB 6.6 MB/s 
     |████████████████████████████████| 596 kB 66.6 MB/s 
     |████████████████████████████████| 6.6 MB 54.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 8.7 MB/s 
     |████████████████████████████████| 140 kB 73.2 MB/s 
     |████████████████████████████████| 1.1 MB 64.2 MB/s 
     |████████████████████████████████| 86 kB 8.3 MB/s 
     |████████████████████████████████|

In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import os
import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from transformers import BertTokenizer, BertModel, BertPreTrainedModel
from datasets import load_dataset, Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from typing import List, Optional, Tuple, Union
import re

In [ ]:
from transformers import AutoTokenizer, AutoModel

In [ ]:
cd /gdrive/MyDrive/Lectures/2022/IntelligentMarketing/content-based

/gdrive/MyDrive/Lectures/2022/IntelligentMarketing/content-based


In [ ]:
#@title 마이너 패키지 로딩
%matplotlib inline
from datetime import datetime
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')

# u.item 로드 및 플롯 수집

../raw/ml-100k
- u.item
    - 0xe9 를 utf-8 디코딩 하지 못하는 것을 처리
        - 'ignore'

In [ ]:
raw_df_ = pd.read_csv('../raw/ml-100k/u.item', sep='|', header=None, encoding_errors='ignore', index_col=False)#, names=['movieId', 'title', 'rel_date','v_rel_date', 'url'])
item_info_df = raw_df_[[0,1,2,4]].copy()
item_info_df.columns=['movieId', 'title', 'release_date', 'imdburl']
item_info_df.set_index('movieId', inplace=True)

In [ ]:
from imdb import Cinemagoer

# create an instance of the Cinemagoer class
ia = Cinemagoer()

# # get a movie
# movie = ia.get_movie('0133093')

# # print the names of the directors of the movie
# print('Directors:')
# for director in movie['directors']:
#     print(director['name'])

# # print the genres of the movie
# print('Genres:')
# for genre in movie['genres']:
#     print(genre)

# # search for a person name
# people = ia.search_person('Mel Gibson')
# for person in people:
#    print(person.personID, person['name'])

## 정확한 매칭

절차
- `search_movie()`: u.item의 타이틀을 입력으로하여 검색
- 다수의 영화 검색 결과 목록중 년도가 동일하고, `plot` 항목이 있는 영화를 선택
- `plot`을 취합

결과
- movie_plots.csv 
    - 123개를 매칭하지 못함
    - 검색 결과가 없거나
    - 동일 년도에 `plot`을 보유한 항목이 없거나

In [ ]:
year_match = re.compile(r'.*[(](\d+)[)]$')

contents = []
for idx, x in tqdm.tqdm(item_info_df['title'].items()):
  movies=ia.search_movie(x)
  m = year_match.match(x)
  content = None
  if m:
    year = m.group(1)
    for small_movie in movies:
      if 'year' in small_movie.keys() and small_movie['year'] == int(year):
        movie=ia.get_movie(small_movie.movieID)
        if 'plot' in movie.keys():
          merged_plot = '\n'.join(movie['plot'])
          content = {'movieId':idx, 'query_title':x, 'title':movie['title'], 'plot': merged_plot}
          break
  if content is None:
    content = {'movieId':idx, 'query_title':x, 'title':x, 'plot': x}
    print(f"Errror: {idx} / {x}")
  
  contents.append(content)

In [ ]:
pd.DataFrame.from_dict(contents).to_csv('movie_plots.csv', index=False)

## 데이터셋과 imdb에 등재된 년도가 다른 경우 보강

위에서 매칭하지 못한 123개의 영화에 대해 보강 작업
- 년도 차이를 1년까지 허용
- contents_modify_2nd.csv
    - 43개를 매칭
    - 80개는 여전히 매칭 못함


In [ ]:
merged_df = pd.read_csv('movie_plots.csv', index_col=False)

In [ ]:
year_match = re.compile(r'.*[(](\d+)[)]$')

contents = []
for idx, x in tqdm.tqdm(merged_df.query("title==plot")['title'].items()):
  movies=ia.search_movie(x)
  m = year_match.match(x)
  content = None
  if m:
    year = m.group(1)
    for small_movie in movies:
      if 'year' in small_movie.keys() and np.abs(small_movie['year']-int(year)) < 2.0:
        movie=ia.get_movie(small_movie.movieID)
        if 'plot' in movie.keys():
          merged_plot = '\n'.join(movie['plot'])
          content = {'movieId':idx, 'query_title':x, 'title':movie['title'], 'plot': merged_plot}
          break
  if content is None:
    content = {'movieId':idx, 'query_title':x, 'title':x, 'plot': x}
    print(f"Errror: {idx} / {x}")
  
  contents.append(content)

In [ ]:
pd.DataFrame.from_dict(contents).to_csv('contents_modify_2nd.csv', index=False)

## 합치기

최종
- movie_plots_80_missings.csv
    - 80개는 매칭하지 못하니, 타이틀을 그대로 plot화 함


In [ ]:
second_df = pd.read_csv('contents_modify_2nd.csv', index_col=False)

In [ ]:
merged_df = pd.read_csv('movie_plots.csv', index_col=False)

In [ ]:
df = pd.merge(merged_df, second_df, how='left', on=['movieId', 'query_title'])
plot = df.plot_x.copy()
plot[df.plot_y.notna()] = df.plot_y[df.plot_y.notna()]
df['plot'] = plot
df = df.drop(columns=['plot_x', 'title_y', 'plot_y'])
df.columns=['moiveId', 'query_title', 'title', 'plot']

In [ ]:
df.to_csv('movie_plots_80_missings.csv', index=False)